In [1]:
from project_path import ROOT_DIR

In [2]:
%%capture
# supress output with %%capture
# change directory
%cd {ROOT_DIR}

In [3]:
# %%capture
# %conda env list
# %conda list
# %dhist
%pip install . --use-feature=in-tree-build

Processing c:\users\vtulus\documents\00-eth_projects\aesamethods
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for aesa-pbs: filename=aesa_pbs-0.8.8-py3-none-any.whl size=124668 sha256=73e66dfaa2cefe1abf0d92e6121c9c090405993752d92ebd4bcdbbb680e21f96
  Stored in directory: C:\Users\vtulus\AppData\Local\Temp\pip-ephem-wheel-cache-cxqtzzm6\wheels\10\fa\a5\382ef7031366baa9086692b52163d123e54538a0a08b665688
Successfully built aesa-pbs
  Attempting uninstall: aesa-pbs
    Found existing installation: aesa-pbs 0.8.8.dev0+premise.aesa
    Uninstalling aesa-pbs-0.8.8.dev0+premise.aesa:
      Successfully uninstalled aesa-pbs-0.8.8.dev0+premise.aesa
Note: you may need to restart the kernel to use updated packages.


In [5]:
# %conda list

In [4]:
import aesa_pbs
aesa_pbs.__version__

'0.8.8'

In [11]:
EXCEL_PATH = Path(
    ROOT_DIR + r"/data/temp" + r"/aesa_ClimateChange_EnergyImbalance.xlsx"
)
# JSON_PATH =  Path(
#     ROOT_DIR + r"/data/temp" + r"/trial.json"
# )
YAML_PATH =  Path(
    ROOT_DIR + r"/data/temp" + r"/trial.yaml"
)
EXCEL_PATH_TRIAL = Path(
    ROOT_DIR + r"/data/temp" + r"/trial.xlsx"
)

In [12]:
convert = aesa_pbs.DataConverter(YAML_PATH)

C:\Users\vtulus\VirtualEnvs\bw2\lib\site-packages\aesa_pbs\utils.py:73: UserWarning: 

Some data are missing in C:\Users\vtulus\Documents\00-ETH_projects\AESAmethods\data\temp\trial.yaml.
Rows with incomplete data are removed.

  warnings.warn(


In [13]:
convert.to_excel()

File created in C:\Users\vtulus\VirtualEnvs\bw2\lib\site-packages\aesa_pbs\data\trial.xlsx


In [8]:
df_ccei = pd.read_excel(EXCEL_PATH)

In [10]:
df_ccei.columns

Index(['name', 'categories', 'amount'], dtype='object')

In [116]:
assert sorted(list(df_ccei.columns)) == sorted(
    ["name", "categories", "amount"]
), "Excel file must contain only 'name', 'categories' and 'amount' columns."

In [72]:
# result = df_ccei.to_json(JSON_PATH, orient="table")
# result = df_ccei.to_json(orient="records", double_precision=15)
result = df_ccei.to_dict(orient="records")

In [66]:
parsed = json.loads(result)

In [67]:
# parsed

In [91]:
class DumperBlankLine(yaml.SafeDumper):
    # HACK: insert blank lines between top-level objects
    # inspired by https://github.com/yaml/pyyaml/issues/127#issuecomment-525800484
    def write_line_break(self, data=None):
        super().write_line_break(data)

        if len(self.indents) == 1:
            super().write_line_break()

In [92]:
with open(YAML_PATH, "w") as file:
    # yaml.safe_dump(
     yaml.dump(
        result,
        file,
        Dumper=DumperBlankLine,
        default_flow_style=False,
        default_style=None,
        sort_keys=False,
        indent=None,
    )
    

In [205]:
with open(YAML_PATH, "r") as file:
    loaded = yaml.safe_load(file)

In [211]:
df_loaded = pd.DataFrame(loaded, columns=["name", "categories", "amount"])

In [212]:
df_loaded

,name,categories,amount
0,"Carbon dioxide, fossil",airos,3.530000e-13
1,"Carbon dioxide, fossil","air::low population density, long-term",3.530000e-13
2,"Carbon dioxide, fossil",air::lower stratosphere + upper troposphere,3.530000e-13
3,"Carbon dioxide, fossil",air::non-urban air or from high stacks,3.530000e-13
4,"Carbon dioxide, fossil",air::urban air close to ground,3.530000e-13
...,...,...,...
93,Sulfur hexafluoride,air,6.710000e-08
94,Sulfur hexafluoride,"air::low population density, long-term",6.710000e-08
95,Sulfur hexafluoride,air::lower stratosphere + upper troposphere,6.710000e-08
96,Sulfur hexafluoride,air::non-urban air or from high stacks,6.710000e-08


In [197]:
df_loaded.fillna("dahdah", inplace=True)

In [214]:
df_loaded.iloc[0,0] = np.nan
df_loaded

,name,categories,amount
0,NaN,airos,3.530000e-13
1,"Carbon dioxide, fossil","air::low population density, long-term",3.530000e-13
2,"Carbon dioxide, fossil",air::lower stratosphere + upper troposphere,3.530000e-13
3,"Carbon dioxide, fossil",air::non-urban air or from high stacks,3.530000e-13
4,"Carbon dioxide, fossil",air::urban air close to ground,3.530000e-13
...,...,...,...
93,Sulfur hexafluoride,air,6.710000e-08
94,Sulfur hexafluoride,"air::low population density, long-term",6.710000e-08
95,Sulfur hexafluoride,air::lower stratosphere + upper troposphere,6.710000e-08
96,Sulfur hexafluoride,air::non-urban air or from high stacks,6.710000e-08


In [185]:
df_loaded.isna().values.any()

True

In [178]:
import warnings

In [215]:
if df_loaded.isna().values.any():
    df_clean = df_loaded.dropna(axis=0, how="any")
    warnings.warn(f"\n\nSome data are missing in {YAML_PATH}.\nRows with incomplete data are removed.\n")
else:
    df_clean = df_loaded
df_clean

C:\Users\vtulus\AppData\Local\Temp\ipykernel_11884\2120660221.py:3: UserWarning: 

Some data are missing in C:\Users\vtulus\Documents\00-ETH_projects\AESAmethods\data\temp\trial.yaml.
Rows with incomplete data are removed.

  warnings.warn(f"\n\nSome data are missing in {YAML_PATH}.\nRows with incomplete data are removed.\n")


,name,categories,amount
1,"Carbon dioxide, fossil","air::low population density, long-term",3.530000e-13
2,"Carbon dioxide, fossil",air::lower stratosphere + upper troposphere,3.530000e-13
3,"Carbon dioxide, fossil",air::non-urban air or from high stacks,3.530000e-13
4,"Carbon dioxide, fossil",air::urban air close to ground,3.530000e-13
5,"Carbon dioxide, from soil or biomass stock",air,3.530000e-13
...,...,...,...
93,Sulfur hexafluoride,air,6.710000e-08
94,Sulfur hexafluoride,"air::low population density, long-term",6.710000e-08
95,Sulfur hexafluoride,air::lower stratosphere + upper troposphere,6.710000e-08
96,Sulfur hexafluoride,air::non-urban air or from high stacks,6.710000e-08


In [210]:
df_clean.empty

True

In [94]:
with pd.ExcelWriter(EXCEL_PATH_TRIAL, mode="w", engine="openpyxl") as writer:
    df_ccei.to_excel(writer, index=False)

In [99]:
EXCEL_PATH_TRIAL.suffix

'.xlsx'

In [216]:
type(EXCEL_PATH_TRIAL)

pathlib.WindowsPath

In [227]:
add = EXCEL_PATH_TRIAL.resolve()

In [235]:
add.stem

'trial'

In [243]:
Path(str(EXCEL_PATH) + f"/{add.stem}.yaml")

(WindowsPath('C:/Users/vtulus/Documents/00-ETH_projects/AESAmethods/data/temp/aesa_ClimateChange_EnergyImbalance.xlsx'),
 WindowsPath('/trial.yaml'))

In [254]:
print(
    f"\n\nSome data are missing in {EXCEL_PATH_TRIAL}."
    "\nRows with incomplete data are removed.\n")



Some data are missing in C:\Users\vtulus\Documents\00-ETH_projects\AESAmethods\data\temp\trial.xlsx.
Rows with incomplete data are removed.

